In [1]:
!python --version

Python 3.10.12


In [2]:
import sys

sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR")

In [3]:
!pip install -qq -r /kaggle/input/monodtr-library-ver2/MonoDTR/requirement.txt
!pip install -qq coloredlogs
!pip install -qq ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.4 MB/s eta 0:00:00


In [4]:
sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR/scripts")
sys.path.append("/kaggle/input/pretrain-monodtr-base")
sys.path.append("/kaggle/input/checkpoint")

In [5]:
import os
import sys
import torch

# Add the directory containing the .so file to the system path
sys.path.append('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn')

# Check if the .so file exists
print(os.listdir('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn'))

# Print Python version
print("Python version:", sys.version)

# Set LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn:' + os.environ.get('LD_LIBRARY_PATH', '')

# Attempt to import the module
try:
    import deform_conv_ext
    print("Import successful!")
except ImportError as e:
    print("Import failed:", e)

# If needed, run the setup or make script
# !python setup.py build
# !python setup.py install
# or
# !bash make.sh


['deform_conv.py', 'deform_conv_ext.cpython-310-x86_64-linux-gnu.so', 'make.sh', 'src', '__init__.py', 'setup.py']
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Import successful!


In [6]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Check your settings.")


CUDA is available!
Number of GPUs available: 1
Current GPU: 0
GPU Name: Tesla P100-PCIE-16GB


In [7]:
# Create a large tensor and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(10000, 10000, device=device)  # This should allocate GPU memory
print("Tensor created on GPU.")
print("Allocated GPU Memory:", torch.cuda.memory_allocated(device))
print("Cached GPU Memory:", torch.cuda.memory_reserved(device))


Tensor created on GPU.
Allocated GPU Memory: 400556032
Cached GPU Memory: 400556032


In [8]:
import torch.nn as nn
import torch
import torch
from torch.nn import Module, Dropout
from einops import rearrange
import torch.nn.functional as F
from ptflops import get_model_complexity_info

In [9]:
# -------------------------- RMSNorm --------------------------
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: (B, L, D)
        norm_x = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return self.scale * norm_x

# -------------------------- FLIP Module --------------------------
def flip(x, dim):
    return torch.flip(x, dims=[dim])

In [10]:
# -------------------------- BiMamba2 Block (Conv Fusion) --------------------------
class BiMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u):
        # u: (B, L, D)
        x = self.linear_x(u)          # (B, L, D)
        bf = self.linear_bf(u)        # (B, L, D)
        bb = self.linear_bb(u)        # (B, L, D)
        z = self.linear_z(u)          # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)         # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)         # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)           # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)           # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)     
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)# (B, D, L)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z
        af = self.norm_f(af)                      # (B, L, D)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))  # (B, D, L)
        ab = F.gelu(ab)                                # (B, D, L)

        z_flip = flip(z, dim=1)                        # (B, L, D)
        ab = self.ssms_b(ab).transpose(1, 2)           # (B, L, D)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip   # (B, D, L)
        ab = self.norm_b(ab)                           # (B, L, D)
        ab = flip(ab, dim=1)                           # (B, L, D)

        out = self.output_linear(af + ab)         # (B, L, D)                 # (B, L, D)
        return out

In [11]:
# -------------------------- CrossMamba2 Module --------------------------
class CrossMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u1, u2):
        # u1: context (B, L, D), u2: depth (B, L, D)
        x = self.linear_x(u1)               # (B, L, D)
        z = self.linear_z(u1)               # (B, L, D)
        bf = self.linear_bf(u2)             # (B, L, D)
        bb = self.linear_bb(u2)             # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)   # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)   # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)     # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)     # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z  # af conv with z
        af = self.norm_f(af)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))
        ab = F.gelu(ab).transpose(1, 2)           # (B, L, D)

        z_flip = flip(z, dim=1)
        ab = self.ssms_b(ab.transpose(1, 2)).transpose(1, 2)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip  # (B, D, L)
        ab = self.norm_b(ab)
        ab = flip(ab, dim=1)

        out = self.output_linear(af + ab)         # (B, L, D)
        return out

In [12]:
# -------------------------- FFN + Norm --------------------------
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)

In [13]:
# -------------------------- Depth Aware Mamba (DTR-Compatible Input) --------------------------
class DepthAwareMamba(nn.Module):
    def __init__(self, output_channel_num):
        super().__init__()
        self.output_channel_num = output_channel_num

        self.encoder_bimamba = BiMamba2(self.output_channel_num)
        self.encoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.encoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.encoder_norm2 = nn.LayerNorm(self.output_channel_num)

        self.decoder_bimamba = BiMamba2(self.output_channel_num)
        self.cross_bimamba = CrossMamba2(self.output_channel_num)
        self.decoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.decoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm2 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm3 = nn.LayerNorm(self.output_channel_num)

    def forward(self, depth_feat, context_feat, depth_pos=None):
        depth_feat = depth_feat.contiguous()
        context_feat = context_feat.contiguous()
        if depth_pos is not None:
            depth_pos = depth_pos.contiguous()
            context_feat = context_feat + depth_pos
    
        # Encoder on context_feat
        x = self.encoder_bimamba(context_feat.contiguous())
        x = self.encoder_norm1((x + context_feat).contiguous())
        x_ffn = self.encoder_ffn(x.contiguous())
        x = self.encoder_norm2((x + x_ffn).contiguous())
    
        # Decoder on depth_feat and fused context
        d = self.decoder_bimamba(depth_feat.contiguous())
        d = self.decoder_norm1((d + depth_feat).contiguous())
    
        x = self.cross_bimamba(x.contiguous(), d.contiguous())
        x = self.decoder_norm2(x.contiguous())
        x_ffn = self.decoder_ffn(x.contiguous())
        x = self.decoder_norm3((x + x_ffn).contiguous())
        return x


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CSA(nn.Module):
    """
    Cross-Scale Attention (Coarse → Less Coarse)
    Used ONLY at low resolution (e.g. 1/32 → 1/16)
    """
    def __init__(self, c):
        super().__init__()
        self.q = nn.Linear(c, c)
        self.k = nn.Linear(c, c)
        self.v = nn.Linear(c, c)
        self.scale = c ** -0.5

    def forward(self, x_fine, x_coarse):
        """
        x_fine   : (N, C, Hf, Wf)
        x_coarse : (N, C, Hc, Wc)
        """
        N, C, Hf, Wf = x_fine.shape
        _, _, Hc, Wc = x_coarse.shape

        q = self.q(x_fine.flatten(2).transpose(1, 2).contiguous())      # (N, HfWf, C)
        k = self.k(x_coarse.flatten(2).transpose(1, 2))    # (N, HcWc, C)
        v = self.v(x_coarse.flatten(2).transpose(1, 2))

        attn = torch.softmax((q @ k.transpose(-2, -1)) * self.scale, dim=-1)
        out = attn @ v                                     # (N, HfWf, C)

        out = out.transpose(1, 2).view(N, C, Hf, Wf)
        return out + x_fine

In [15]:
class MSR(nn.Module):
    """
    Multi-Scale Refinement (Top-Down FPN-style)
    """
    def __init__(self, d=256):
        super().__init__()
        self.refine = nn.Sequential(
            nn.Conv2d(d, d, 3, padding=1, bias=False),
            nn.BatchNorm2d(d),
            nn.ReLU(inplace=True)
        )

    def forward(self, coarse, fine):
        """
        coarse: (N, d, H/16, W/16)
        fine  : (N, d, H/8,  W/8)
        """
        up = F.interpolate(coarse, size=fine.shape[2:], mode='nearest')
        return self.refine(up + fine)

In [16]:
class DepthAwareFEv2(nn.Module):
    """
    Depth-Aware Feature Enhancement v2
    - Learn depth confidence
    - Feature gating
    - Soft depth embedding
    """
    def __init__(self, dim):
        super().__init__()

        self.conf_head = nn.Sequential(
            nn.Conv2d(dim, dim // 2, 3, padding=1, bias=False),
            nn.BatchNorm2d(dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim // 2, 1, 1),
            nn.Sigmoid()
        )

        self.refine = nn.Sequential(
            nn.Conv2d(dim, dim, 3, padding=1, bias=False),
            nn.BatchNorm2d(dim),
            nn.ReLU(inplace=True)
        )

        self.token_proj = nn.Linear(dim, dim)

    def forward(self, x):
        """
        x: (N, C, H, W)
        """
        N, C, H, W = x.shape

        conf = self.conf_head(x)           # (N,1,H,W)
        gated = x * conf
        depth_feat = self.refine(gated)    # (N,C,H,W)

        depth_token = depth_feat.flatten(2).transpose(1, 2).contiguous()
        depth_emb = self.token_proj(depth_token)

        return depth_feat, conf, depth_emb

In [17]:
class BottomUpFusion(nn.Module):
    def __init__(self, d=256):
        super().__init__()
        self.down = nn.Sequential(
            nn.Conv2d(d, d, 2, stride=2, bias=False),
            nn.BatchNorm2d(d),
            nn.ReLU(inplace=True)
        )

    def forward(self, f_1_4, f_1_8, depth_conf):
        """
        f_1_4   : (N, d, H/4, W/4)
        f_1_8   : (N, d, H/8, W/8)
        depth_conf : (N,1,H/8,W/8)
        """
        down = self.down(f_1_4)            # (N,d,H/8,W/8)
        gated = down * depth_conf          # depth-aware gating
        return f_1_8 + gated


In [18]:
from visualDet3D.networks.backbones.dla import dla102
from visualDet3D.networks.backbones.dlaup import DLAUp
from visualDet3D.networks.detectors.dfe import DepthAwareFE
from visualDet3D.networks.detectors.dpe import DepthAwarePosEnc
from visualDet3D.networks.detectors.dtr import DepthAwareTransformer
import math
import time
import csv

In [19]:
import timm

class MonoDTRCore(nn.Module):
    def __init__(self):
        super().__init__()

        # Backbone
        self.backbone = timm.create_model(
            'swin_tiny_patch4_window7_224',
            pretrained=True,
            features_only=True,
            img_size=(288, 1280)
        )

        # Projection
        self.proj_1_4  = nn.Conv2d(96, 256, 1)
        self.proj_1_8  = nn.Conv2d(192, 256, 1)
        self.proj_1_16 = nn.Conv2d(384, 256, 1)
        self.proj_1_32 = nn.Conv2d(768, 256, 1)

        # Backbone fusion
        self.csa_32_16 = CSA(256)          # coarse depth hint
        self.msr_16_8  = MSR(256)          # top-down

        # Depth-aware feature enhancement
        self.dfe = DepthAwareFEv2(256)
        self.dpe = DepthAwarePosEnc(256)

        # Bottom-up (depth-guided)
        self.bottom_up = BottomUpFusion(256)

        # Transformer
        self.dtr = DepthAwareTransformer(256)

        self.img_conv = nn.Conv2d(256, 256, 3, padding=1)

    def forward(self, x):
        img = x['image']

        # Swin features
        feats = self.backbone(img)
        feats = [f.permute(0, 3, 1, 2) for f in feats]

        f1_4  = self.proj_1_4(feats[0])    # (N,256,H/4,W/4)
        f1_8  = self.proj_1_8(feats[1])    # (N,256,H/8,W/8)
        f1_16 = self.proj_1_16(feats[2])   # (N,256,H/16,W/16)
        f1_32 = self.proj_1_32(feats[3])   # (N,256,H/32,W/32)

        # Coarse depth context
        f1_16 = self.csa_32_16(f1_16, f1_32)

        # Top-down refinement
        f1_8_td = self.msr_16_8(f1_16, f1_8)

        # Depth-aware enhancement (learn depth confidence here)
        depth_feat, depth_conf, depth_emb = self.dfe(f1_8_td)

        # Bottom-up gated fusion (local detail injection)
        f1_8_final = self.bottom_up(f1_4, f1_8_td, depth_conf)

        # Depth positional encoding
        N, C, H, W = f1_8_final.shape
        depth_emb = self.dpe(depth_emb, (H, W))

        # Image feature
        img_feat = (f1_8_final + self.img_conv(f1_8_final)) \
                    .flatten(2).transpose(1, 2)

        # DTR
        feat = self.dtr(
            depth_feat.flatten(2).transpose(1, 2),
            img_feat,
            depth_emb
        )
        feat = feat.transpose(1, 2).view(N, C, H, W)

        return feat, depth_conf


In [20]:
from visualDet3D.networks.heads.detection_3d_head import AnchorBasedDetection3DHead
from visualDet3D.networks.heads.depth_losses import bin_depths, DepthFocalLoss
from visualDet3D.networks.utils.registry import DETECTOR_DICT

In [21]:
class MonoDTR(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.mono_core = MonoDTRCore()
        self.bbox_head = AnchorBasedDetection3DHead(**cfg.head)

    def train_forward(self, images, annotations, P2, depth_gt=None):
        features, depth_conf = self.mono_core(dict(image=images, P2=P2))

        cls_preds, reg_preds = self.bbox_head(
            dict(features=features, image=images, P2=P2)
        )

        anchors = self.bbox_head.get_anchor(images, P2)
        cls_loss, reg_loss, loss_dict = self.bbox_head.loss(
            cls_preds, reg_preds, anchors, annotations, P2
        )

        # Light regularization for depth confidence
        conf_loss = depth_conf.mean()
        loss_dict['conf_loss'] = conf_loss

        return cls_loss, reg_loss, conf_loss, loss_dict

    def test_forward(self, images, P2):
        features, _ = self.mono_core(dict(image=images, P2=P2))
        cls_preds, reg_preds = self.bbox_head(
            dict(features=features, image=images, P2=P2)
        )
        anchors = self.bbox_head.get_anchor(images, P2)
        return self.bbox_head.get_bboxes(cls_preds, reg_preds, anchors, P2, images)

    def forward(self, inputs):
        if isinstance(inputs, list) and len(inputs) >= 3:
            return self.train_forward(*inputs)
        else:
            return self.test_forward(*inputs)


In [22]:
import os
import sys
import numpy as np
from easydict import EasyDict
from tqdm import tqdm
from fire import Fire
import coloredlogs
import logging
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from _path_init import *
from visualDet3D.networks.utils.registry import DETECTOR_DICT, DATASET_DICT, PIPELINE_DICT
from visualDet3D.networks.utils.utils import BackProjection, BBox3dProjector, get_num_parameters
from visualDet3D.evaluator.kitti.evaluate import evaluate
import visualDet3D.data.kitti.dataset
from visualDet3D.utils.timer import Timer
from visualDet3D.utils.utils import LossLogger, cfg_from_file
from visualDet3D.networks.optimizers import optimizers, schedulers

In [23]:
import torch
from torch.cuda.amp import autocast  # For AMP
import torch.optim as optim
from visualDet3D.utils.utils import LossLogger
from visualDet3D.utils.utils import compound_annotation

def My_train_mono_detection(data, module: nn.Module,
                         optimizer: optim.Optimizer,
                         writer: SummaryWriter = None,
                         loss_logger: LossLogger = None,
                         global_step: int = None,
                         epoch_num: int = None,
                         cfg: EasyDict = EasyDict(),
                         iter_num: int = None):
    if iter_num is None:
        iter_num = 0  # Fallback, but manage externally
    steps = cfg.trainer.accumulation_steps
    is_accum_start = (iter_num % steps == 0)
    if is_accum_start and optimizer is not None:
        optimizer.zero_grad(set_to_none=True)  # Faster than zero_grad()

    images, P2, labels, bbox2d, bbox_3d, depth = data
    # Assume batch=1, so lists of len=1

    # Handle empty annotations gracefully
    max_length = np.max([len(label) for label in labels]) if labels else 0
    if max_length == 0:
        return  # Still increments iter_num externally

    annotation = compound_annotation(labels, max_length, bbox2d, bbox_3d, cfg.obj_types)
    annotation_tensor = images.new(annotation).cuda()  # Inherit dtype/device

    # Forward pass with AMP for efficiency
    # with autocast(device_type='cuda', enabled=cfg.get('use_amp', True)):
    cls_loss, reg_loss, conf_loss, loss_dict = module([
        images.cuda().float().contiguous(),
        annotation_tensor,
        P2.cuda(),
        depth.cuda().contiguous()
    ])
    cls_loss = cls_loss.mean()
    reg_loss = reg_loss.mean()
    total_loss = cls_loss + reg_loss + 0.1 * conf_loss
    # loss_dict['total_loss'] = total_loss.item()  # Add for logging

    # Log unscaled micro-batch losses
    if loss_logger is not None:
        loss_logger.update(loss_dict)
    del loss_dict

    if optimizer is not None:
        # Check for invalid losses
        if torch.isnan(total_loss) or torch.isinf(total_loss):
            print(f"Warning: Invalid loss at iter {iter_num}: {total_loss.item()}")
            # Optionally: optimizer.zero_grad(); return
            total_loss = torch.tensor(0.0, device=total_loss.device, requires_grad=True)

        scaled_loss = total_loss / steps
        scaled_loss.backward()

    # Update every accumulation_steps
    if (iter_num + 1) % steps == 0:
        # Gradient clipping
        if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
            torch.nn.utils.clip_grad_norm_(module.parameters(), cfg.optimizer.clipped_gradient_norm)
        else:
            print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)  # Reset after step for next cycle

In [24]:
def set_requires_grad(model, keywords, requires_grad):
    for name, param in model.named_parameters():
        if any(k in name for k in keywords):
            param.requires_grad = requires_grad

In [25]:
def configure_stage(detector, stage):
    # Freeze all first
    for _, p in detector.named_parameters():
        p.requires_grad = False

    # Stage 0: sanity
    if stage == 0:
        set_requires_grad(detector, ["msr_16_8", "dfe", "bbox_head"], True)

    # Stage 1: detection warmup
    elif stage == 1:
        set_requires_grad(detector, [
            "msr_16_8", "bottom_up", "dfe", "dtr", "bbox_head"
        ], True)

    # Stage 2: depth-aware refinement
    elif stage == 2:
        set_requires_grad(detector, [
            "csa_32_16", "msr_16_8", "bottom_up", "dfe", "dtr", "bbox_head"
        ], True)

        # unfreeze backbone stage 3–4
        for name, p in detector.named_parameters():
            if "backbone.layers.2" in name or "backbone.layers.3" in name:
                p.requires_grad = True

    # Stage 3: full finetune
    elif stage == 3:
        for _, p in detector.named_parameters():
            p.requires_grad = True


In [26]:
def build_optimizer(detector, stage):
    param_groups = []

    if stage >= 2:
        param_groups.append({
            "params": detector.mono_core.backbone.parameters(),
            "lr": 1e-5
        })

    param_groups += [
        {"params": detector.mono_core.csa_32_16.parameters(), "lr": 2e-5},
        {"params": detector.mono_core.msr_16_8.parameters(), "lr": 1e-4},
        {"params": detector.mono_core.bottom_up.parameters(), "lr": 1e-4},
        {"params": detector.mono_core.dfe.parameters(), "lr": 1e-4},
        {"params": detector.mono_core.dtr.parameters(), "lr": 1e-4},
        {"params": detector.bbox_head.parameters(), "lr": 1e-4},
    ]

    return torch.optim.AdamW(param_groups, weight_decay=0.01)

In [27]:
def configure_stage(detector, stage):
    # Freeze all first
    for _, p in detector.named_parameters():
        p.requires_grad = False

    # Stage 0: sanity
    if stage == 0:
        # set_requires_grad(detector, ["msr_16_8", "dfe", "bbox_head"], True)
        set_requires_grad(detector, ["msr_16_8", "dfe"], True)

    # Stage 1: detection warmup
    elif stage == 1:
        set_requires_grad(detector, [
            "msr_16_8", "bottom_up", "dfe"
            # "msr_16_8", "bottom_up", "dfe", "dtr", "bbox_head"
        ], True)

    # Stage 2: depth-aware refinement
    elif stage == 2:
        set_requires_grad(detector, [
            "csa_32_16", "msr_16_8", "bottom_up", "dfe", "dtr", "bbox_head"
        ], True)

        # unfreeze backbone stage 3–4
        for name, p in detector.named_parameters():
            if "backbone.layers.2" in name or "backbone.layers.3" in name:
                p.requires_grad = True

    # Stage 3: full finetune
    elif stage == 3:
        for _, p in detector.named_parameters():
            p.requires_grad = True

In [28]:
from collections import OrderedDict  # Add this import
import matplotlib.pyplot as plt
import cv2
from skimage.measure import block_reduce
def main(config="/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py", experiment_name="default", world_size=1, local_rank=-1, stage = 0, resume_ckpt=None):

    ## Get config
    cfg = cfg_from_file(config)

    ## Collect distributed(or not) information
    cfg.dist = EasyDict()
    cfg.dist.world_size = world_size
    cfg.dist.local_rank = local_rank
    is_distributed = local_rank >= 0 # local_rank < 0 -> single training
    is_logging     = local_rank <= 0 # only log and test with main process
    is_evaluating  = local_rank <= 0

    ## Setup writer if local_rank > 0
    recorder_dir = os.path.join(cfg.path.log_path, experiment_name + f"config={config}")
    if is_logging: # writer exists only if not distributed and local rank is smaller
        ## Clean up the dir if it exists before
        if os.path.isdir(recorder_dir):
            os.system("rm -r {}".format(recorder_dir))
            print("clean up the recorder directory of {}".format(recorder_dir))
        writer = SummaryWriter(recorder_dir)

        ## Record config object using pprint
        import pprint

        formatted_cfg = pprint.pformat(cfg)
        writer.add_text("config.py", formatted_cfg.replace(' ', '&nbsp;').replace('\n', '  \n')) # add space for markdown style in tensorboard text
    else:
        writer = None

    ## Set up GPU and distribution process
    if is_distributed:
        cfg.trainer.gpu = local_rank # local_rank will overwrite the GPU in configure file
    gpu = min(cfg.trainer.gpu, torch.cuda.device_count() - 1)
    torch.backends.cudnn.benchmark = getattr(cfg.trainer, 'cudnn', False)
    torch.cuda.set_device(gpu)
    if is_distributed:
        torch.distributed.init_process_group(backend='nccl', init_method='env://')
    print(local_rank)
    
    BATCH_SIZE = {
        0: 4,
        1: 2,
        2: 2,
        3: 2
    }
    ## define datasets and dataloader.
    dataset_train = DATASET_DICT[cfg.data.train_dataset](cfg)
    dataset_val = DATASET_DICT[cfg.data.val_dataset](cfg, "validation")

    dataloader_train = DataLoader(dataset_train, num_workers=cfg.data.num_workers,
                                  batch_size=BATCH_SIZE[stage], collate_fn=dataset_train.collate_fn, shuffle=local_rank<0, drop_last=True,
                                  sampler=torch.utils.data.DistributedSampler(dataset_train, num_replicas=world_size, rank=local_rank, shuffle=True) if local_rank >= 0 else None)

    detector = MonoDTR(cfg.detector)         
            
    ## Convert to cuda
    if is_distributed:
        detector = torch.nn.SyncBatchNorm.convert_sync_batchnorm(detector)
        detector = torch.nn.parallel.DistributedDataParallel(detector.cuda(), device_ids=[gpu], output_device=gpu)
    else:
        detector = detector.cuda()
    
    ## define optimizer and weight decay
    # optimizer = optimizers.build_optimizer(cfg.optimizer, detector)

    # ## define scheduler
    # scheduler_config = getattr(cfg, 'scheduler', None)
    # scheduler = schedulers.build_scheduler(scheduler_config, optimizer)

    configure_stage(detector, stage)
    
    optimizer = build_optimizer(detector, stage)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=cfg.trainer.max_epochs
    )

    scheduler_config = getattr(cfg, 'scheduler', None)
    is_iter_based = getattr(scheduler_config, "is_iter_based", False)

    ## define loss logger
    training_loss_logger =  LossLogger(writer, 'train') if is_logging else None

    ## training pipeline
    if 'training_func' in cfg.trainer:
        # training_dection = PIPELINE_DICT[cfg.trainer.training_func] #hoanbi1
        training_dection = My_train_mono_detection
    else:
        raise KeyError

    ## Get evaluation pipeline
    if 'evaluate_func' in cfg.trainer:
        evaluate_detection = PIPELINE_DICT[cfg.trainer.evaluate_func]
        print("Found evaluate function {}".format(cfg.trainer.evaluate_func))
    else:
        evaluate_detection = None
        print("Evaluate function not found")


    ## timer is used to estimate eta
    timer = Timer()

    if resume_ckpt is not None:
        state = torch.load(resume_ckpt, map_location="cpu")
        detector.load_state_dict(state, strict=False)
        print("Loaded model weights (optimizer reset)")
    else:
        state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_location='cpu')
        new_state_dict = OrderedDict()
        loaded_keys = []  # Track which parameters are loaded
        for k, v in state_dict_pretrain.items():
            if k in detector.state_dict() and v.shape == detector.state_dict()[k].shape:
                new_state_dict[k] = v
                loaded_keys.append(k)  # Record loaded keys
                print(f"Parameter {k} is loaded from pretrain")
    
        detector.load_state_dict(new_state_dict, strict=False)
    
    for name, param in detector.named_parameters():
        if param.requires_grad == True:
            print(f"Parameter {name} is trainable")
            
    ## Record basic information of the model
    if is_logging:
        string1 = detector.__str__().replace(' ', '&nbsp;').replace('\n', '  \n')
        writer.add_text("model structure", string1) # add space for markdown style in tensorboard text
        num_parameters = get_num_parameters(detector)
        print(f'number of trained parameters of the model: {num_parameters}')
        
    detector.train()
    global_step = 0
    
    l_rate = optimizer.param_groups[0]['lr']
    print(f"Starting LR = {l_rate:.6f}")
    
    # evaluate_detection(cfg, detector.module if is_distributed else detector, dataset_val, writer, 0)
    
    STAGE_EPOCHS = {
        0: 40,
        1: 15,
        2: 30,
        3: 10
    }

    # for epoch_num in range(cfg.trainer.max_epochs):
    # for epoch_num in range(epoch, (40+epoch)):
    for epoch_num in range(STAGE_EPOCHS[stage]):
        ## Start training for one epoch
        torch.cuda.empty_cache()
        detector.train()
        if training_loss_logger:
            training_loss_logger.reset()
        for iter_num, data in enumerate(dataloader_train):
            training_dection(data, detector, optimizer, writer, training_loss_logger, global_step, epoch_num, cfg, iter_num)

            global_step += 1
            if (iter_num + 1) % cfg.trainer.accumulation_steps == 0:
                if is_iter_based:
                    scheduler.step()
    
                if is_logging and global_step % cfg.trainer.disp_iter == 0:
                    ## Log loss, print out and write to tensorboard in main process
                    if 'total_loss' not in training_loss_logger.loss_stats:
                        print(f"\nIn epoch {epoch_num}, iteration:{iter_num}, global_step:{global_step}, total_loss not found in logger.")
                    else:
                        log_str = 'Epoch: {} | Iteration: {}  | Running loss: {:1.5f} | eta:{}'.format(
                            epoch_num, iter_num, training_loss_logger.loss_stats['total_loss'].avg,
                            timer.compute_eta(global_step, len(dataloader_train) * cfg.trainer.max_epochs))
                        print(log_str, end='\r')
                        writer.add_text("training_log/train", log_str, global_step)
                        training_loss_logger.log(global_step)
                        
        # Handle partial accumulation at end of epoch
        if (iter_num + 1) % cfg.trainer.accumulation_steps != 0:
            # Gradient clipping if applicable
            if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
                torch.nn.utils.clip_grad_norm_(detector.parameters(), cfg.optimizer.clipped_gradient_norm)
            else:
                print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            
        if not is_iter_based:
            scheduler.step()
            
        l_rate = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch_num+1} — LR = {l_rate:.6f}")
        
        ## save model in main process if needed
        if is_logging:
            torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                cfg.path.checkpoint_path, '{}_latest.pth'.format(
                    cfg.detector.name)
                )
            )
        if is_logging and (epoch_num + 1) % cfg.trainer.save_iter == 0:
            torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                cfg.path.checkpoint_path, '{}_{}.pth'.format(
                    cfg.detector.name,epoch_num)
                )
            )
        checkpoint = {
                "epoch": epoch_num + 1,
                "global_step": global_step,
                "state_dict_backbone": detector.state_dict(),
                "state_optimizer": optimizer.state_dict(),
                "state_lr_scheduler": scheduler.state_dict()
            }
        torch.save(checkpoint, os.path.join(cfg.path.checkpoint_path, f"checkpoint_resume.pt"))
        print(f"Save checkpoint at epoch {epoch_num+1} successfully!")
        ## test model in main process if needed
        if is_evaluating and evaluate_detection is not None and cfg.trainer.test_iter > 0 and (epoch_num + 1) % cfg.trainer.test_iter == 0:
            print("\n/**** start testing after training epoch {} ******/".format(epoch_num))
            evaluate_detection(cfg, detector.module if is_distributed else detector, dataset_val, writer, epoch_num)
            print("/**** finish testing after training epoch {} ******/".format(epoch_num))

        torch.save(
            detector.state_dict(),
            f"{cfg.path.checkpoint_path}/stage{stage}_epoch{epoch_num}.pth"
        )

        if is_distributed:
            torch.distributed.barrier() # wait untill all finish a epoch

        if is_logging:
            writer.flush()

In [29]:
%%writefile my_config.py

from easydict import EasyDict as edict
import os 
import numpy as np

cfg = edict()
cfg.obj_types = ['Car']
#cfg.obj_types = ['Car', 'Pedestrian', 'Cyclist']

## trainer
trainer = edict(
    gpu = 0,
    max_epochs = 100, 
    disp_iter = 100,
    save_iter = 5,
    test_iter = 5,
    training_func = "train_mono_detection",
    test_func = "test_mono_detection",
    evaluate_func = "evaluate_kitti_obj",
    accumulation_steps = 4,  # hoanbi1
)

cfg.trainer = trainer

## path
path = edict()
path.data_path = "/kaggle/input/kitti-3d-object-detection-dataset/training" # used in visualDet3D/data/.../dataset
path.test_path = "/kaggle/input/kitti-3d-object-detection-dataset/testing" # used in visualDet3D/data/.../dataset
path.visualDet3D_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D" # The path should point to the inner subfolder
path.project_path = "/kaggle/working/" # or other path for pickle files, checkpoints, tensorboard logging and output files.
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)
path.project_path = os.path.join(path.project_path, 'MonoDTR')
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)

path.log_path = os.path.join(path.project_path, "log")
if not os.path.isdir(path.log_path):
    os.mkdir(path.log_path)

path.checkpoint_path = os.path.join(path.project_path, "checkpoint")
if not os.path.isdir(path.checkpoint_path):
    os.mkdir(path.checkpoint_path)

path.preprocessed_path = os.path.join(path.project_path, "output")
if not os.path.isdir(path.preprocessed_path):
    os.mkdir(path.preprocessed_path)

path.train_imdb_path = os.path.join(path.preprocessed_path, "training")
if not os.path.isdir(path.train_imdb_path):
    os.mkdir(path.train_imdb_path)

path.val_imdb_path = os.path.join(path.preprocessed_path, "validation")
if not os.path.isdir(path.val_imdb_path):
    os.mkdir(path.val_imdb_path)

path.test_imdb_path = os.path.join(path.preprocessed_path, "test")
if not os.path.isdir(path.test_imdb_path):
    os.mkdir(path.test_imdb_path)

path.test_result_path = os.path.join(path.test_imdb_path, "data")
if not os.path.isdir(path.test_result_path):
    os.mkdir(path.test_result_path)

cfg.path = path

## optimizer
optimizer = edict(
    type_name = 'adam',
    keywords = edict(
        lr        = 1e-5, #Cái này dùng khi train full model 
        # lr        = 1e-4, #Cái này dùng khi freeze model
        weight_decay = 0,
    ),
    clipped_gradient_norm = 0.1
)
cfg.optimizer = optimizer
## scheduler
scheduler = edict(
    type_name = 'CosineAnnealingLR',
    keywords = edict(
        T_max     = cfg.trainer.max_epochs,
        eta_min   = 5e-6,
    )
)
cfg.scheduler = scheduler

## data
data = edict(
    batch_size = 1,
    num_workers = 8,
    rgb_shape = (288, 1280, 3),
    train_dataset = "KittiMonoDataset",
    val_dataset   = "KittiMonoDataset",
    test_dataset  = "KittiMonoTestDataset",
    train_split_file = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'train.txt'),
    val_split_file   = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'val.txt'),
)

data.augmentation = edict(
    rgb_mean = np.array([0.485, 0.456, 0.406]),
    rgb_std  = np.array([0.229, 0.224, 0.225]),
    cropSize = (data.rgb_shape[0], data.rgb_shape[1]),
    crop_top = 100,
)
data.train_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='PhotometricDistort', keywords=edict(distort_prob=1.0, contrast_lower=0.5, contrast_upper=1.5, saturation_lower=0.5, saturation_upper=1.5, hue_delta=18.0, brightness_delta=32)),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='RandomMirror', keywords=edict(mirror_prob=0.5)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
data.test_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
cfg.data = data

## networks
detector = edict()
detector.obj_types = cfg.obj_types
detector.name = 'MonoDTR'
detector.mono_backbone=edict(
    msr_loss_weight = 1.0,  # Weight cho loss mới
    lambda_bal = 0.85,       # Từ paper
)
head_loss = edict(
    fg_iou_threshold = 0.5,
    bg_iou_threshold = 0.4,
    L1_regression_alpha = 5 ** 2,
    focal_loss_gamma = 2.0,
    balance_weight   = [20.0],
    #balance_weight   = [20.0, 40, 40],
    regression_weight = [1, 1, 1, 1, 1, 1, 12, 1, 1, 0.5, 0.5, 0.5, 1], #[x, y, w, h, cx, cy, z, sin2a, cos2a, w, h, l]
)
head_test = edict(
    score_thr=0.75,
    cls_agnostic = False,
    nms_iou_thr=0.4,
    post_optimization=False
)

anchors = edict(
        {
            'obj_types': cfg.obj_types,
            'pyramid_levels':[3],
            'strides': [2 ** 3],
            'sizes' : [24],
            'ratios': np.array([0.5, 1, 2.0]),
            'scales': np.array([2 ** (i / 4.0) for i in range(16)]),
        }
    )

head_layer = edict(
    num_features_in=256,
    num_cls_output=len(cfg.obj_types)+1,
    num_reg_output=12,
    cls_feature_size=256,
    reg_feature_size=256,
)
detector.head = edict(
    num_regression_loss_terms=13,
    preprocessed_path=path.preprocessed_path,
    num_classes     = len(cfg.obj_types),
    anchors_cfg     = anchors,
    layer_cfg       = head_layer,
    loss_cfg        = head_loss,
    test_cfg        = head_test
)
detector.anchors = anchors
detector.loss = head_loss
cfg.detector = detector


Writing my_config.py


In [30]:
def run_training(stage = 0, resume_ckpt = None):
    # config_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py"  # Path to your config file
    config_path = "/kaggle/working/my_config.py"  # Path to your config file
    experiment_name = "EXP_NAME"  # Use the defined experiment name
    world_size = 1  # For single GPU training
    local_rank = 0  # Local rank set to 0 as per your command
    %env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
    main(config=config_path, experiment_name=experiment_name, world_size=world_size, stage = stage, resume_ckpt = resume_ckpt)

In [31]:
run_training(stage = 0)

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
-1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Found evaluate function evaluate_kitti_obj


<ipython-input-28-531be45e46b5>:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_lo

...

Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:0.16, 0.17, 0.21
bev  AP:0.00, 0.01, 0.01
3d   AP:0.00, 0.00, 0.00
aos  AP:0.08, 0.09, 0.12
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:0.16, 0.17, 0.21
bev  AP:0.07, 0.12, 0.06
3d   AP:0.01, 0.02, 0.02
aos  AP:0.08, 0.09, 0.12

/**** finish testing after training epoch 29 ******/
Epoch 31 — LR = 0.000016
Save checkpoint at epoch 31 successfully!
Epoch 32 — LR = 0.000015
Save checkpoint at epoch 32 successfully!
Epoch 33 — LR = 0.000015
Save checkpoint at epoch 33 successfully!
Epoch 34 — LR = 0.000015
Save checkpoint at epoch 34 successfully!
Epoch 35 — LR = 0.000015
Save checkpoint at epoch 35 successfully!

/**** start testing after training epoch 34 ******/
clean up the recorder directory of /kaggle/working/MonoDTR/output/validation/data
rebuild /kaggle/working/MonoDTR/output/validation/data


100%|██████████| 3769/3769 [04:26<00:00, 14.14it/s]
/usr/local/lib/python3.10/dist-packages/numba_cuda/numba/cuda/dispatcher.py:579: NumbaPerformanceWarning: Grid size 18 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba_cuda/numba/cuda/dispatcher.py:579: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba_cuda/numba/cuda/dispatcher.py:579: NumbaPerformanceWarning: Grid size 20 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:0.19, 0.20, 0.23
bev  AP:0.00, 0.01, 0.01
3d   AP:0.00, 0.00, 0.00
aos  AP:0.11, 0.12, 0.13
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:0.19, 0.20, 0.23
bev  AP:0.07, 0.05, 0.05
3d   AP:0.01, 0.02, 0.02
aos  AP:0.11, 0.12, 0.13

/**** finish testing after training epoch 34 ******/
Epoch 36 — LR = 0.000014
Save checkpoint at epoch 36 successfully!
Epoch 37 — LR = 0.000014
Save checkpoint at epoch 37 successfully!
Epoch 38 — LR = 0.000014
Save checkpoint at epoch 38 successfully!
Epoch 39 — LR = 0.000013
Save checkpoint at epoch 39 successfully!
Epoch 40 — LR = 0.000013
Save checkpoint at epoch 40 successfully!

/**** start testing after training epoch 39 ******/
clean up the recorder directory of /kaggle/working/MonoDTR/output/validation/data
rebuild /kaggle/working/MonoDTR/output/validation/data


100%|██████████| 3769/3769 [04:22<00:00, 14.33it/s]


Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:0.08, 0.13, 0.15
bev  AP:0.00, 0.01, 0.01
3d   AP:0.00, 0.00, 0.00
aos  AP:0.05, 0.09, 0.10
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:0.08, 0.13, 0.15
bev  AP:0.06, 0.04, 0.05
3d   AP:0.01, 0.02, 0.02
aos  AP:0.05, 0.09, 0.10

/**** finish testing after training epoch 39 ******/
